In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [2]:
test = pd.read_csv('./datasets/test.csv')

See which columns have missing data and how much is missing

In [3]:
missing = {}
for col in test.columns:
    n = test[col].isnull().sum()
    if n > 0:
        missing[col] = n
missing

{'Lot Frontage': 160,
 'Alley': 821,
 'Mas Vnr Type': 1,
 'Mas Vnr Area': 1,
 'Bsmt Qual': 25,
 'Bsmt Cond': 25,
 'Bsmt Exposure': 25,
 'BsmtFin Type 1': 25,
 'BsmtFin Type 2': 25,
 'Electrical': 1,
 'Fireplace Qu': 422,
 'Garage Type': 44,
 'Garage Yr Blt': 45,
 'Garage Finish': 45,
 'Garage Qual': 45,
 'Garage Cond': 45,
 'Pool QC': 875,
 'Fence': 707,
 'Misc Feature': 838}

In [4]:
#Filling in certain values based on what ismost common using group by
test.loc[(test['Garage Qual'].isnull()==True) & (test['Garage Type']=='Detchd'), 'Garage Qual'] = 'TA'
test.loc[(test['Garage Cond'].isnull()==True) & (test['Garage Type']=='Detchd'), 'Garage Cond'] = 'TA'
test.loc[(test['Garage Finish'].isnull()==True) & (test['Garage Type']=='Detchd'), 'Garage Finish'] = 'Unf'
test.loc[(test['Garage Yr Blt'].isnull()==True) & (test['Garage Type']=='Detchd'), 'Garage Yr Blt'] = 1910

#replacing values where NA has meaning
test['Alley'].replace(np.nan,value='No Alley Access',inplace=True)
test['Bsmt Qual'].replace(np.nan,value='None',inplace=True)
test['Bsmt Cond'].replace(np.nan,value='No Basement',inplace=True)
test['Bsmt Exposure'].replace(np.nan,value='None',inplace=True)
test['BsmtFin Type 1'].replace(np.nan,value='No Basement',inplace=True)
test['BsmtFin Type 2'].replace(np.nan,value='No Basement',inplace=True)
test['Mas Vnr Type'].replace(np.nan,value='None',inplace=True)
test['Mas Vnr Area'].replace(np.nan,value=0,inplace=True)
test['Fireplace Qu'].replace(np.nan,value='None',inplace=True)
test['Fence'].replace(np.nan,value='No Fence',inplace=True)
test['Misc Feature'].replace(np.nan,value='None',inplace=True)
test['Pool QC'].replace(np.nan,value='No Pool',inplace=True)
test['Garage Type'].replace(np.nan,value='No Garage',inplace=True)
test['Garage Yr Blt'].replace(np.nan,value='No Garage',inplace=True)
test['Garage Finish'].replace(np.nan,value='No Garage',inplace=True)
test['Garage Qual'].replace(np.nan,value='No Garage',inplace=True)
test['Garage Cond'].replace(np.nan,value='No Garage',inplace=True)
test['Electrical'].replace(np.nan,value='SBrkr',inplace=True)

Using views to determine what values are actaully missing and which ones are just place holders for information  
Outputs supressed for readibility

In [5]:
test['Garage Type'].replace(np.nan,value='No Garage',inplace=True)

In [6]:
view = test[test['Garage Yr Blt'].isnull()==True]
view[['Year Built','Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Qual', 'Garage Cond']];

In [7]:
test.groupby('Garage Type')['Garage Finish'].value_counts();

In [8]:
test.groupby('Garage Type')['Garage Finish'].value_counts();

In [9]:
test.groupby('Garage Type')['Garage Qual'].value_counts();

In [10]:
test.groupby('Garage Type')['Garage Cond'].value_counts();

In [11]:
view2 = test[test['Electrical'].isnull()==True]
view2[['Year Built']];

In [12]:
test.groupby('Year Built')['Electrical'].value_counts();

In [13]:
#getting same regression model
ames = pd.read_csv('./datasets/train.csv')
ames['logfoot'] = np.log(ames['Lot Frontage'])
ames['logarea'] = np.log(ames['Lot Area'])
lot_train = ames[ames['Lot Frontage'].isnull() == False]
lr = LinearRegression()
X = lot_train[['logarea']]
y = lot_train['logfoot']
lr.fit(X,y);

In [14]:
test['logfoot'] = np.log(test['Lot Frontage'])
test['logarea'] = np.log(test['Lot Area'])
test_train = test[test['Lot Frontage'].isnull() == False]
test_null = test[test['Lot Frontage'].isnull() == True]

In [15]:
test_null['logfoot'] = lr.predict(test_null[['logarea']])

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
test = pd.concat([test_train,test_null])

In [17]:
test['Lot Frontage'] = np.exp(test['logfoot'])

In [18]:
test['Lot Frontage'].isnull().sum()

0

In [19]:
test.drop(columns=['logfoot','logarea'], inplace=True)

In [20]:
test.to_csv('cleaned_test.csv')